## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Mataura,NZ,-46.19,168.86,55.11,65,99,10.65,overcast clouds
1,1,Hermanus,ZA,-34.42,19.23,39.99,92,85,3.18,light rain
2,2,Coruripe,BR,-10.13,-36.18,74.95,81,74,12.77,broken clouds
3,3,Chokurdakh,RU,70.63,147.92,41.32,85,100,4.68,overcast clouds
4,4,Khislavichi,RU,54.19,32.16,62.15,75,90,13.42,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 10
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Mataura,NZ,-46.19,168.86,55.11,65,99,10.65,overcast clouds
1,1,Hermanus,ZA,-34.42,19.23,39.99,92,85,3.18,light rain
2,2,Coruripe,BR,-10.13,-36.18,74.95,81,74,12.77,broken clouds
3,3,Chokurdakh,RU,70.63,147.92,41.32,85,100,4.68,overcast clouds
4,4,Khislavichi,RU,54.19,32.16,62.15,75,90,13.42,overcast clouds
5,5,Katsuura,JP,35.13,140.30,86.00,84,40,9.17,scattered clouds
6,6,Punta Arenas,CL,-53.15,-70.92,39.20,86,40,29.97,scattered clouds
7,7,Husavik,IS,66.04,-17.34,46.40,87,75,10.29,broken clouds
8,8,Necochea,AR,-38.55,-58.74,43.00,61,0,5.01,clear sky
9,9,Andros Town,BS,24.70,-77.77,86.00,66,20,10.29,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Mataura,NZ,-46.19,168.86,55.11,65,99,10.65,overcast clouds
1,1,Hermanus,ZA,-34.42,19.23,39.99,92,85,3.18,light rain
2,2,Coruripe,BR,-10.13,-36.18,74.95,81,74,12.77,broken clouds
3,3,Chokurdakh,RU,70.63,147.92,41.32,85,100,4.68,overcast clouds
4,4,Khislavichi,RU,54.19,32.16,62.15,75,90,13.42,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
712,712,Hofn,IS,64.25,-15.21,50.32,75,97,1.68,overcast clouds
713,713,Arman,RU,59.70,150.17,53.80,51,0,9.51,clear sky
714,714,Kendallville,US,41.44,-85.27,73.40,56,1,5.82,clear sky
716,716,Jacareacanga,BR,-6.22,-57.75,83.55,56,83,2.26,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Mataura,NZ,55.11,overcast clouds,-46.19,168.86,
1,Hermanus,ZA,39.99,light rain,-34.42,19.23,
2,Coruripe,BR,74.95,broken clouds,-10.13,-36.18,
3,Chokurdakh,RU,41.32,overcast clouds,70.63,147.92,
4,Khislavichi,RU,62.15,overcast clouds,54.19,32.16,
5,Katsuura,JP,86.00,scattered clouds,35.13,140.30,
6,Punta Arenas,CL,39.20,scattered clouds,-53.15,-70.92,
7,Husavik,IS,46.40,broken clouds,66.04,-17.34,
8,Necochea,AR,43.00,clear sky,-38.55,-58.74,
9,Andros Town,BS,86.00,few clouds,24.70,-77.77,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Mataura,NZ,55.11,overcast clouds,-46.19,168.86,Ellie's Villa
1,Hermanus,ZA,39.99,light rain,-34.42,19.23,Misty Waves Boutique Hotel
2,Coruripe,BR,74.95,broken clouds,-10.13,-36.18,Pousada e Hotel Litoral Sul
3,Chokurdakh,RU,41.32,overcast clouds,70.63,147.92,
4,Khislavichi,RU,62.15,overcast clouds,54.19,32.16,Stolik Na Gorke
...,...,...,...,...,...,...,...
712,Hofn,IS,50.32,overcast clouds,64.25,-15.21,Hótel Höfn
713,Arman,RU,53.80,clear sky,59.70,150.17,
714,Kendallville,US,73.40,clear sky,41.44,-85.27,Best Western Kendallville Inn
716,Jacareacanga,BR,83.55,broken clouds,-6.22,-57.75,Casa do Ian😎


In [14]:
# 8a. Create the output File (CSV)
output_data_file = hotel_df.to_csv('WeatherPy_vacation.csv')
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

'City_ID,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name\n0,Mataura,NZ,55.11,overcast clouds,-46.19,168.86,Ellie\'s Villa\n1,Hermanus,ZA,39.99,light rain,-34.42,19.23,Misty Waves Boutique Hotel\n2,Coruripe,BR,74.95,broken clouds,-10.13,-36.18,Pousada e Hotel Litoral Sul\n3,Chokurdakh,RU,41.32,overcast clouds,70.63,147.92,\n4,Khislavichi,RU,62.15,overcast clouds,54.19,32.16,Stolik Na Gorke\n5,Katsuura,JP,86.0,scattered clouds,35.13,140.3,Katsuura Hotel Mikazuki\n6,Punta Arenas,CL,39.2,scattered clouds,-53.15,-70.92,Hotel Dreams Del Estrecho\n7,Husavik,IS,46.4,broken clouds,66.04,-17.34,Bjarnabúð\n8,Necochea,AR,43.0,clear sky,-38.55,-58.74,dyd -mar Hotel\n9,Andros Town,BS,86.0,few clouds,24.7,-77.77,Hanks place\n10,Butaritari,KI,83.48,scattered clouds,3.07,172.79,Isles Sunset Lodge\n11,Havelock,US,84.0,moderate rain,34.88,-76.9,Sherwood Motel\n12,Ushuaia,AR,33.8,light snow,-54.8,-68.3,Albatros Hotel\n13,Krasnoselkup,RU,54.18,overcast clouds,65.7,82.47,Krasnosel\'kupnefteprod

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))